# Real-Time predict without metawindow

In [1]:
import cv2
import dlib
from scipy.spatial import distance
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import joblib

In [2]:
# Función para calcular EAR
def calculate_ear(eye_points):
    A = distance.euclidean(eye_points[1], eye_points[5])
    B = distance.euclidean(eye_points[2], eye_points[4])
    C = distance.euclidean(eye_points[0], eye_points[3])
    ear = (A + B) / (2.0 * C)
    return ear

In [3]:
# Función para calcular MAR
def calculate_mar(mouth_points):
    A = distance.euclidean(mouth_points[2], mouth_points[8])
    B = distance.euclidean(mouth_points[3], mouth_points[7])
    C = distance.euclidean(mouth_points[4], mouth_points[6])
    D = distance.euclidean(mouth_points[0], mouth_points[5])
    mar = (A + B + C) / (2.0 * D)
    return mar

In [4]:
# Función para estimar ángulo de cabeza
def calculate_head_angle(landmarks):
    model_points = np.array([
        (0.0, 0.0, 0.0),  # Nariz
        (0.0, -330.0, -65.0),  # Barbilla
        (-225.0, 170.0, -135.0),  # Ojo izquierdo
        (225.0, 170.0, -135.0),  # Ojo derecho
        (-150.0, -150.0, -125.0),  # Boca izquierda
        (150.0, -150.0, -125.0)  # Boca derecha
    ])
    image_points = np.array([
        (landmarks.part(30).x, landmarks.part(30).y),  # Nariz
        (landmarks.part(8).x, landmarks.part(8).y),  # Barbilla
        (landmarks.part(36).x, landmarks.part(36).y),  # Ojo izquierdo
        (landmarks.part(45).x, landmarks.part(45).y),  # Ojo derecho
        (landmarks.part(48).x, landmarks.part(48).y),  # Boca izquierda
        (landmarks.part(54).x, landmarks.part(54).y)  # Boca derecha
    ], dtype="double")
    
    focal_length = 640  # Aproximado para webcam
    center = (640 / 2, 480 / 2)
    camera_matrix = np.array([[focal_length, 0, center[0]],
                             [0, focal_length, center[1]],
                             [0, 0, 1]], dtype="double")
    dist_coeffs = np.zeros((4, 1))
    _, rotation_vector, _ = cv2.solvePnP(model_points, image_points, camera_matrix, dist_coeffs)
    angle = np.linalg.norm(rotation_vector) * 180 / np.pi
    return angle

In [5]:
# Función para contar parpadeos
def count_blinks(ears, ear_threshold=0.2):
    blinks = 0
    prev_ear = ears[0]
    for ear in ears[1:]:
        if prev_ear > ear_threshold and ear <= ear_threshold:
            blinks += 1
        prev_ear = ear
    return blinks

In [6]:
# Cargar detector y predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("face_landmarks/shape_predictor_68_face_landmarks.dat")

# Cargar modelos
rf_model = joblib.load('models_without_metawindow/rf_model.pkl')
svm_model = joblib.load('models_without_metawindow/svm_model.pkl')
scaler = joblib.load('models_without_metawindow/scaler.pkl')

# Prueba en tiempo real
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: No se pudo abrir la cámara")
    exit()

EAR_THRESHOLD = 0.2
CONSECUTIVE_FRAMES = 20
fatigue_counter = 0
window_features = []

while True:
    ret, frame = cap.read()
    if not ret:
        break
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)
    for face in faces:
        landmarks = predictor(gray, face)
        left_eye = [(landmarks.part(i).x, landmarks.part(i).y) for i in range(36, 42)]
        right_eye = [(landmarks.part(i).x, landmarks.part(i).y) for i in range(42, 48)]
        mouth = [(landmarks.part(i).x, landmarks.part(i).y) for i in range(48, 68)]
        ear = (calculate_ear(left_eye) + calculate_ear(right_eye)) / 2.0
        mar = calculate_mar(mouth)
        head_angle = calculate_head_angle(landmarks)
        window_features.append({'ear': ear, 'mar': mar, 'head_angle': head_angle})
        
        if len(window_features) >= 20:
            window_ears = [f['ear'] for f in window_features]
            window_mars = [f['mar'] for f in window_features]
            window_angles = [f['head_angle'] for f in window_features]
            blink_freq = count_blinks(window_ears)
            features = {
                'ear_mean': np.mean(window_ears),
                'ear_std': np.std(window_ears),
                'ear_min': np.min(window_ears),
                'mar_mean': np.mean(window_mars),
                'mar_std': np.std(window_mars),
                'mar_max': np.max(window_mars),
                'head_angle_mean': np.mean(window_angles),
                'blink_freq': blink_freq
            }
            X_real_time = scaler.transform([list(features.values())])
            prediction = rf_model.predict(X_real_time)[0]  # Usa RF por defecto
            if prediction == 1:
                fatigue_counter += 1
                if fatigue_counter >= CONSECUTIVE_FRAMES:
                    cv2.putText(frame, "ALERTA: FATIGA DETECTADA", (10, 30),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            else:
                fatigue_counter = 0
            window_features = window_features[1:]  # Mantener ventana deslizante
            cv2.putText(frame, f"EAR: {ear:.2f} MAR: {mar:.2f} Angle: {head_angle:.2f}", (10, 60),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    
    cv2.imshow("Driver Fatigue Detection", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

c:\Users\AGpar\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
c:\Users\AGpar\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
c:\Users\AGpar\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
c:\Users\AGpar\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
c:\Users\AGpar\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
c:\Users\AGpar\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature nam